<a href="https://colab.research.google.com/github/BatmarcNYU/dataScienceBootCamp/blob/main/ML_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

df = pd.read_csv('/content/employee.csv')

# df.shape #(4277, 13)
df = df.drop(columns=['id', 'timestamp', 'country'])
df.loc[df['hours_per_week'].isna(), 'hours_per_week'] = df['hours_per_week'].median()
df.loc[df['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = df['telecommute_days_per_week'].median()
df = df.dropna()

# df['country'] = pd.Categorical(df['country'])
# df['country_code'] = df['country'].cat.codes
training_data = df.copy()
for feat in ['is_manager', 'certifications']:
  training_data[feat] = training_data[feat].replace(to_replace=['Yes'], value=1)
  training_data[feat] = training_data[feat].replace(to_replace=['No'], value=1)
countries = ['Slovenia', 'United States', 'Sweden', 'United Kingdom', 'Canada', 'New Zealand', 'Belgium', 'France', 'Australia', 'India', 'Denmark', 'Romania', 'Poland', 'Norway', 'Croatia', 'Netherlands', 'Hungary', 'Argentina', 'Costa Rica', 'Switzerland', 'Germany', 'Mexico', 'Ukraine', 'Spain', 'Czech Republic', 'Portugal', 'South Africa', 'Hong Kong', 'Russia', 'Ireland', 'Guernsey', 'Israel', 'Bulgaria', 'Uganda', 'Finland', 'Italy', 'Jersey', 'United Arab Emirates', 'Austria', 'Turkey', 'Bahrain', 'Greece', 'Colombia', 'Kenya', 'Peru', 'Saudi Arabia', 'Albania', 'Iceland', 'Guatemala', 'Belarus', 'Moldova', 'Puerto Rico', 'Brazil', 'Indonesia', 'Slovakia', 'Serbia and Montenegro', 'Singapore', 'Malta', 'Venezuela', 'Latvia', 'China', 'Ecuador', 'Pakistan', 'Vietnam', 'Bolivia', 'Paraguay', 'Thailand', 'Lithuania', 'Jordan', 'Macedonia', 'Malaysia', 'Luxembourg', 'Philippines', 'Syria', 'Ghana', 'Taiwan', 'Estonia', 'Uruguay']

# for country in df['country']:
#   training_data['country_code'] = training_data['country'].replace(to_replace=[country], value=countries.index(country))
# training_data = training_data.drop(columns=['country'])
categorical_feats = [feat for feat in training_data.columns if training_data[feat].dtype == 'object' and feat not in ['is_manager', 'certifications']]

# x = training_data[categorical_feats]
# x
# training_data['country_code']
# df['country_code'] = df['country'].astype(int)
# training_data[['is_manager', 'certifications']]
final_data = pd.get_dummies(training_data, columns=categorical_feats, drop_first=True, dtype=int)

y = final_data['salary']
X = final_data.drop(columns=['salary'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

num_cols = ['job_years','hours_per_week','telecommute_days_per_week']

# before
# mae  9406.270120188927
# mse  12812.500292380955
# Z-score method for numerical columns to remove outliers
z_scores = (df[num_cols] - df[num_cols].mean()) / df[num_cols].std()
outliers_zscore = df[(z_scores.abs() > 3).any(axis=1)]

df = df.drop(outliers_zscore.index)
# after
# mae  8046.525820478533
# mse  9405.74164031662
# # IQR method
# Q1 = df[num_cols].quantile(0.25)
# Q3 = df[num_cols].quantile(0.75)
# IQR = Q3 - Q1
# outliers_iqr = df[((df[num_cols] < (Q1 - 1.5 * IQR)) | (df[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# # Remove outliers from the DataFrame
# df = df[~((df[num_cols] < (Q1 - 1.5 * IQR)) | (df[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# after
# mae  9767.047123321843
# mse  13759.637455681834

scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

reg=LinearRegression()
reg.fit(X_train, y_train)

# reg.coef_
# reg.intercept_

mean_squared_error(y_train, reg.predict(X_train))/np.mean(y_train)

y_pred = reg.predict(X_test)
print(y_pred)

mae = mean_absolute_error(y_test, y_pred)
print('mae ', mae)

mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print('mse ', mse)


[15073.52504568 18052.82609652 17141.87156964 ... 17355.74617264
 17171.29983574 17530.77859363]
mae  10057.19848602559
mse  14570.558482521576
